In [1]:
import joblib
import json
import os
import pandas as pd

from flask import Flask
from flask import jsonify
from flask import request
# from flask_restful import Api, Resource
import sys

os.chdir('/home/jupyter/tfg-sb-meal-delivery-prediction/')

from src.data.data_collect import read_train_data
from src.model.xgboost_model import preprocess_data, train_xgboost_model

In [2]:
df_train = read_train_data()

In [3]:
with open('./src/test/test_api_train.json') as f:
    config = json.load(f)

In [4]:
config

{'center_id': 55, 'meal_id': 1993}

In [13]:
# Preprocess the dataframe
center_id = config['center_id']
meal_id = config['meal_id']

In [9]:
center_id = 10
meal_id = 1062

In [14]:
df_preprocessed = preprocess_data(df_train, center_id, meal_id)

/opt/conda/lib/python3.7/site-packages/pandas/core/generic.py:5159: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self[name] = value
/home/jupyter/tfg-sb-meal-delivery-prediction/src/model/xgboost_model.py:19: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_processed['day'] = df_processed.date.dt.day
/home/jupyter/tfg-sb-meal-delivery-prediction/src/model/xgboost_model.py:20: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] 

In [21]:
df_preprocessed.columns

Index(['week', 'checkout_price', 'base_price', 'emailer_for_promotion',
       'homepage_featured', 'num_orders', 'city_code', 'region_code',
       'op_area', 'day', 'month', 'year', 'quarter'],
      dtype='object')

In [15]:
features = list(df_preprocessed.drop(columns='num_orders').columns)

In [16]:
regressor_model, rmse = train_xgboost_model(df_preprocessed)

RMSE: 45.64661830504204


In [17]:
result = {
    'features' : features,
    'rmse' : rmse
}

In [18]:
result

{'features': ['week',
  'checkout_price',
  'base_price',
  'emailer_for_promotion',
  'homepage_featured',
  'city_code',
  'region_code',
  'op_area',
  'day',
  'month',
  'year',
  'quarter'],
 'rmse': 45.64661830504204}

In [19]:
features

['week',
 'checkout_price',
 'base_price',
 'emailer_for_promotion',
 'homepage_featured',
 'city_code',
 'region_code',
 'op_area',
 'day',
 'month',
 'year',
 'quarter']

# Test predict API

In [23]:
from src.data.data_collect import read_test_data

In [24]:
df_test = read_test_data()

In [25]:
df_test_processed = preprocess_data(df_test, center_id, meal_id)

/opt/conda/lib/python3.7/site-packages/pandas/core/generic.py:5159: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self[name] = value
/home/jupyter/tfg-sb-meal-delivery-prediction/src/model/xgboost_model.py:19: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_processed['day'] = df_processed.date.dt.day
/home/jupyter/tfg-sb-meal-delivery-prediction/src/model/xgboost_model.py:20: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] 

In [26]:
import requests

In [27]:
API_ENDPOINT = 'http://localhost:5000/test'

In [28]:
r = requests.get(url = API_ENDPOINT)

In [29]:
r.json()

{'response': 'test successful!!!!!'}

In [32]:
row = json.loads(df_test_processed.reset_index().to_json(orient='records'))

In [33]:
API_ENDPOINT = 'http://localhost:5000/predict'

In [34]:
r = requests.post(url = API_ENDPOINT, json=row)

In [35]:
r

<Response [200]>

In [36]:
r.json()

[{'date': 1483228800000, 'num_orders': 122.3300476074},
 {'date': 1483833600000, 'num_orders': 123.3151168823},
 {'date': 1484438400000, 'num_orders': 122.3300476074},
 {'date': 1485043200000, 'num_orders': 122.3300476074},
 {'date': 1485648000000, 'num_orders': 122.3300476074},
 {'date': 1486252800000, 'num_orders': 121.9145507812},
 {'date': 1486857600000, 'num_orders': 122.8962936401},
 {'date': 1487462400000, 'num_orders': 121.9145507812},
 {'date': 1488067200000, 'num_orders': 121.9145507812},
 {'date': 1488672000000, 'num_orders': 121.7965927124}]

# Useful links
+ [Curl rest API](https://linuxize.com/post/curl-rest-api/)
+ [Python-How to send a dataframe to external API (push)](https://stackoverflow.com/questions/63035336/python-how-to-send-a-dataframe-to-external-api-push)
+ [Python Requests post() Method](https://www.w3schools.com/python/ref_requests_post.asp)
+ [Return two reponses in Flask/JavaScript](https://stackoverflow.com/questions/39460769/return-two-responses-in-flask-javascript)